In [1]:
import keras
keras.__version__

/home/farid/.local/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


'2.1.5'

# Text generation with LSTM

This notebook contains the code samples found in Chapter 8, Section 1 of [Deep Learning with Python](https://www.manning.com/books/deep-learning-with-python?a_aid=keras&a_bid=76564dff). Note that the original text features far more content, in particular further explanations and figures: in this notebook, you will only find source code and related comments.

----

[...]

## Implementing character-level LSTM text generation


Let's put these ideas in practice in a Keras implementation. The first thing we need is a lot of text data that we can use to learn a 
language model. You could use any sufficiently large text file or set of text files -- Wikipedia, the Lord of the Rings, etc. In this 
example we will use some of the writings of Nietzsche, the late-19th century German philosopher (translated to English). The language model 
we will learn will thus be specifically a model of Nietzsche's writing style and topics of choice, rather than a more generic model of the 
English language.

## Preparing the data

Let's start by downloading the corpus and converting it to lowercase:

In [5]:
import keras
import numpy as np

path = keras.utils.get_file(
    'nietzsche.txt',
    origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
text = open(path).read().lower()
print('Corpus length:', len(text))
print(text[:200])

Corpus length: 600893
preface


supposing that truth is a woman--what then? is there not ground
for suspecting that all philosophers, in so far as they have been
dogmatists, have failed to understand women--that the terrib



Next, we will extract partially-overlapping sequences of length `maxlen`, one-hot encode them and pack them in a 3D Numpy array `x` of 
shape `(sequences, maxlen, unique_characters)`. Simultaneously, we prepare a array `y` containing the corresponding targets: the one-hot 
encoded characters that come right after each extracted sequence.

In [9]:
maxlen = 60 # Length of extracted character sequences
step = 3 # We sample a new sequence every `step` characters
sentences = [] # This holds our extracted sequences
next_chars = [] # This holds the targets (the follow-up characters)

for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
    
print('Number of sequences:', len(sentences))

# List of unique characters in the corpus
chars = sorted(list(set(text)))
print('Unique characters:', len(chars))

# Dictionary mapping unique characters to their index in `chars`
char_indices = {char: chars.index(char) for char in chars}
print('char_indices', char_indices)

# Next, one-hot encode the characters into binary arrays.
print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool) #3D tensor
y = np.zeros((len(sentences), len(chars)), dtype=np.bool) #2D tensor

for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

Number of sequences: 200278
Unique characters: 57
char_indices {'\n': 0, ' ': 1, '!': 2, '"': 3, "'": 4, '(': 5, ')': 6, ',': 7, '-': 8, '.': 9, '0': 10, '1': 11, '2': 12, '3': 13, '4': 14, '5': 15, '6': 16, '7': 17, '8': 18, '9': 19, ':': 20, ';': 21, '=': 22, '?': 23, '[': 24, ']': 25, '_': 26, 'a': 27, 'b': 28, 'c': 29, 'd': 30, 'e': 31, 'f': 32, 'g': 33, 'h': 34, 'i': 35, 'j': 36, 'k': 37, 'l': 38, 'm': 39, 'n': 40, 'o': 41, 'p': 42, 'q': 43, 'r': 44, 's': 45, 't': 46, 'u': 47, 'v': 48, 'w': 49, 'x': 50, 'y': 51, 'z': 52, 'ä': 53, 'æ': 54, 'é': 55, 'ë': 56}
Vectorization...


## Building the network

Our network is a single `LSTM` layer followed by a `Dense` classifier and softmax over all possible characters. But let us note that 
recurrent neural networks are not the only way to do sequence data generation; 1D convnets also have proven extremely successful at it in 
recent times.

In [10]:
from keras import layers

model = keras.models.Sequential()
model.add(layers.LSTM(128, input_shape=(maxlen, len(chars))))
model.add(layers.Dense(len(chars), activation='softmax'))

Since our targets are one-hot encoded, we will use `categorical_crossentropy` as the loss to train the model:

In [11]:
optimizer = keras.optimizers.RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

## Training the language model and sampling from it


Given a trained model and a seed text snippet, we generate new text by repeatedly:

* 1) Drawing from the model a probability distribution over the next character given the text available so far
* 2) Reweighting the distribution to a certain "temperature"
* 3) Sampling the next character at random according to the reweighted distribution
* 4) Adding the new character at the end of the available text

This is the code we use to reweight the original probability distribution coming out of the model, 
and draw a character index from it (the "sampling function"):

In [13]:
np.random.multinomial(1, [1/6.]*6, size=1)

array([[0, 0, 1, 0, 0, 0]])

In [14]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)


Finally, this is the loop where we repeatedly train and generated text. We start generating text using a range of different temperatures 
after every epoch. This allows us to see how the generated text evolves as the model starts converging, as well as the impact of 
temperature in the sampling strategy.

In [21]:
import random
import sys

for epoch in range(1, 60):
    print('epoch', epoch)
    # Fit the model for 1 epoch on the available training data
    model.fit(x, y,
              batch_size=128,
              epochs=1)

    # Select a text seed at random
    start_index = random.randint(0, len(text) - maxlen - 1)
    generated_text = text[start_index: start_index + maxlen]
    print('--- Generating with seed: "' + generated_text + '"')

    for temperature in [0.2, 0.5, 1.0, 1.2]:
        print('------ temperature:', temperature)
        sys.stdout.write(generated_text)

        # We generate 400 characters
        for i in range(400):
            sampled = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(generated_text):
                sampled[0, t, char_indices[char]] = 1.

            preds = model.predict(sampled, verbose=2)
            #print('preds', sum(preds[0]))
            preds = preds[0]
            next_index = sample(preds, temperature)
            next_char = chars[next_index]

            generated_text += next_char
            generated_text = generated_text[1:]

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

epoch 1
Epoch 1/1
200278/200278 [==============================] - 211s 1ms/step - loss: 1.3830
--- Generating with seed: "ecause
they cannot appreciate, and are very ready to admire "
------ temperature: 0.2
ecause
they cannot appreciate, and are very ready to admire and the self-concerned to the self-concerned to the subtle the suffered to the soul and and the states and the superit the present to the such a person to a proved to the subject the words of the subject of the problem of the standard and the soul of the superit of the self-such as the man" and the present to the self-concerned to the experience of the subject of the subject of the standard to the
------ temperature: 0.5
erience of the subject of the subject of the standard to the evil-perhaps let us have the surmander to we soul the sublimed the history of the virtue and deceive to the ear indeer the delicate of the taste. it is all most the moral, and that is still play
stofulation of the great taste of the self-suffere

wheat thoue all say begre, to sorvely
fact an a purpose some heart
have uppir. in the groking. it is
thimo-"sunners (this drestroushedful for utdeded, were peracpechiest conscious in
itsireles
and the macked that extent, and condeitoo, apparalur of depends of life
grown phenolosedig, a sa culture of unfathbous still the daws wit
epoch 5
Epoch 1/1
200278/200278 [==============================] - 223s 1ms/step - loss: 1.3609
--- Generating with seed: "arentness of such
ownership, and wishes to have finer tests "
------ temperature: 0.2
arentness of such
ownership, and wishes to have finer tests of the moral surprusting of the form of the feeling of the complate of the sense of the future of the sense of the future, and the soul, the moral surprosing of the more surembs the feeling of the sense of the complexice than the states of the complexice that is the more form of the same as the sense of the spirit of the one in the fact that the most comparison of the struggle of the problem of th

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: RuntimeWarning: divide by zero encountered in log
  This is separate from the ipykernel package so we can avoid doing imports until


able and
occass becomes an us. but his peraming and willand ofor the certain and greeked at acts modern iderence of first present gloor, a theseous. fina, out-or in vrymet refare wis se
------ temperature: 1.2
sent gloor, a theseous. fina, out-or in vrymet refare wis seldopherim cumidger
strengs bodified. iotificl to hard, whene unme suffer), impresudanitylers from this
aesome in chremon to "upon these faum of cuts. is becund, what founds or had causays, but fiese
closeness:ancus,
and
by chroun
argusk of epyer. but none from pud" is innocent, effectible, fulth loors, foll-extrived himteldy to anything so ensig: has eary
onighs. "for my call a , 
or what takes. m
epoch 7
Epoch 1/1
200278/200278 [==============================] - 227s 1ms/step - loss: 1.3523
--- Generating with seed: "kill a fly intentionally
without thinking very much about it"
------ temperature: 0.2
kill a fly intentionally
without thinking very much about its and the contine that is a discreading to the spirit of the

he morality of the fact that there is a continux to the same there are as the comprehent, and man must and possession seemselity of the religious and the contemncial contemnes to the case of the same determine with its very the fact that is acts and the course of recertand to all the spirits--and out of themselves with the child strong of the enough so so instrument:--there is assumpiticiline pewary have been distrustful belief of the masters, the moral th
------ temperature: 1.0
ry have been distrustful belief of the masters, the moral thought is all vequent? them out their philtke; to it.

i
 churable of dwarfitious dir valuc of
abaritious should away to his agains man be
much we produtor," to dofry of succeed discerte and the rals"ing of depresiders whe have to many the influences are always been ? stover force of
the resides themselves, it was not frou-dack. notforiandknement humanil line--has around. flone which this back and 
------ temperature: 1.2
knement humanil line--has arou

...

f which him.ele.=--the antiposous of trailined, and best formingous exercise one and partly furtain european interesting, an ares: for their degenerating efbits, quite anose exercise before this now, is
good implacain
to getten man, our
evil"er. to the motive (and
occositched with the compasting agre,
asomedy
of the enoretingering of myset acts also
they better interrotative very hopeation touh the skepps incomplasthe. howerbily . the riths with creative q
epoch 45
Epoch 1/1
200278/200278 [==============================] - 371s 2ms/step - loss: 1.3109
--- Generating with seed: "t--and this is all that could be wished--inasmuch as it
cann"
------ temperature: 0.2
t--and this is all that could be wished--inasmuch as it
cannot better the state of the present and spirit of the same and morality, the ear and the expedient of the same of the philosophers of the spirit of the spirit of the place of the spirit of the habit of the demonstly in the same of the will and the sense of the contrary 

on[th userely genium as stand. not hoidwapbficl, experion a desconden to
the hand as these hrin anything bodl
heavy taking sufficient the morality.=--but bearistruce, utility,
in
order they would have upon exist). the end'eds, precisely intolupitionally rath which he venture him desire prige and epocty rigilable pe
------ temperature: 1.2
th which he venture him desire prige and epocty rigilable perceive; "boter dirmelase that oft. to
verfrient to the serves necess
for intercosement them, sigbared bodk diudes,
his hoped
impermanical valiking forhersedment" the judgment the-but later, of the frenchhe, "miffer, we delusion;
pitumes flow, ilsome metaphysical philosopher, new love repudiated discomble
own need who
among lade; foith-upon, philosophical and glenatiquilly but for the sense of sign
epoch 49
Epoch 1/1
200278/200278 [==============================] - 251s 1ms/step - loss: 1.3220
--- Generating with seed: "an, the helpful man, is duly styled "good". (at first other
"
------ tempe

n of the strength to seems to seems to the superiority of the considerately better, and has to the postimating of man sought and sanddhered and the one man we will to success were does not realmily theregments of commander, the tempo of man who all something of good ambition, and been the considerations, assomed to consequence, and for the reasons and not of the school. thereby and in the sublimates of the duty of the ethic astricting and symbolous for the
------ temperature: 1.0
es of the duty of the ethic astricting and symbolous for thereby, who tenem: be sympathy and case this to be "seems the developed and
tenching: by means of all new to advantlemely of manpjultuponical
higher: the
christian ariuges, but into elye and kind, and why
"worker flight of the will. this late,--what namiluts, everyone virtut".--this soul-but a worthing ((because done in the interest wish
to un que tud; for gequenter, in whatards, ideas our
fully gev
------ temperature: 1.2
 un que tud; for gequenter, in

whereby such natures may resist the general extent of the constate and strongest which the same and the present conscience of the same of the same of the all things of the same of the consideration of the same and comprehend of the same and caste of the same and conscience of the same and conception of the same and possible to the continuely of the same and caste that it is a man and also the same that the same in the same and say and also t
------ temperature: 0.5
d also the same that the same in the same and say and also the present contrady and the weaker conscience, and
appreciate and as the incernation of the stand!" in the sense of the same and say to thinking for the same things, the words of the self-profuld and motives and say to comed and in the same world in the fact and not generated with regard to his lack of the "free of the conception of the rank of the extranges of the most expression of the sentiment
------ temperature: 1.0
ank of the extranges of the most expression o

is above nature. "the reer ave everywhodely even of a such evbavirations. he stand who has a noble itshapid and man--has a



As you can see, a low temperature results in extremely repetitive and predictable text, but where local structure is highly realistic: in 
particular, all words (a word being a local pattern of characters) are real English words. With higher temperatures, the generated text 
becomes more interesting, surprising, even creative; it may sometimes invent completely new words that sound somewhat plausible (such as 
"eterned" or "troveration"). With a high temperature, the local structure starts breaking down and most words look like semi-random strings 
of characters. Without a doubt, here 0.5 is the most interesting temperature for text generation in this specific setup. Always experiment 
with multiple sampling strategies! A clever balance between learned structure and randomness is what makes generation interesting.

Note that by training a bigger model, longer, on more data, you can achieve generated samples that will look much more coherent and 
realistic than ours. But of course, don't expect to ever generate any meaningful text, other than by random chance: all we are doing is 
sampling data from a statistical model of which characters come after which characters. Language is a communication channel, and there is 
a distinction between what communications are about, and the statistical structure of the messages in which communications are encoded. To 
evidence this distinction, here is a thought experiment: what if human language did a better job at compressing communications, much like 
our computers do with most of our digital communications? Then language would be no less meaningful, yet it would lack any intrinsic 
statistical structure, thus making it impossible to learn a language model like we just did.


## Take aways

* We can generate discrete sequence data by training a model to predict the next tokens(s) given previous tokens.
* In the case of text, such a model is called a "language model" and could be based on either words or characters.
* Sampling the next token requires balance between adhering to what the model judges likely, and introducing randomness.
* One way to handle this is the notion of _softmax temperature_. Always experiment with different temperatures to find the "right" one.